LeNet-5 Prac

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [5]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2 ,0.2))
])

train_data = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)
test_data = torchvision.datasets.CIFAR10(root="./data",train=False, download=True, transform=test_transform)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


LeNet-5 model

In [10]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64*6*6, 64)
        self.fc2 = nn.Linear(64,10)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x))) #feat map >> 14 * 14
        x = self.pool(torch.relu(self.conv2(x))) #feat map >> 6 * 6
        
        x = torch.flatten(x, 1) #flatten -> 64 * 6 * 6
        x = self.fc2(torch.relu(self.fc1(x))) #64, 10
        
        return x

모델 학습 및 평가 함수 구현

In [11]:
def train_and_eval(model, n_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(n_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            imgs, labels = data
            optimizer.zero_grad()
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            if i% 200 == 190:
                print('[%d, %5d] loss : %.3f' % (epoch +1, i+1, running_loss / 200))
                running_loss = 0.0
                
    print("Finished Training")
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            imgs, labels = data
            outputs = model(imgs)
            _, predicted =torch.max(outputs.data , 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    acc = 100 * correct / total
    print("Acc >> %.2f %%"%acc)

In [12]:
print("LeNet-5")
model = LeNet()
train_and_eval(model, 5)

LeNet-5
[1,   191] loss : 2.146
[1,   391] loss : 2.047
[1,   591] loss : 1.911
[2,   191] loss : 1.633
[2,   391] loss : 1.645
[2,   591] loss : 1.591
[3,   191] loss : 1.475
[3,   391] loss : 1.509
[3,   591] loss : 1.481
[4,   191] loss : 1.373
[4,   391] loss : 1.417
[4,   591] loss : 1.381
[5,   191] loss : 1.286
[5,   391] loss : 1.330
[5,   591] loss : 1.307
Finished Training
Acc >> 54.20 %
